In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install cvxpy

     |████████████████████████████████| 993kB 1.8MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 215kB 5.5MB/s eta 0:00:01
     |████████████████████████████████| 3.6MB 6.0MB/s eta 0:00:01     |████████████████▋               | 1.8MB 6.0MB/s eta 0:00:01
     |████████████████████████████████| 153kB 13.9MB/s eta 0:00:01
  Created wheel for cvxpy: filename=cvxpy-1.1.1-cp36-cp36m-linux_x86_64.whl size=2498580 sha256=98d3574c6892eead00276ad5bc261d0c64df419ecaec247badefa17dfd6c3c05
  Stored in directory: /home/nbuser/.cache/pip/wheels/06/db/59/b5af93d86703e0903b9b94ccc300ac70daf9d273f13e6c0350
Successfully built cvxpy
  Created wheel for scs: filename=scs-2.1.2-cp36-cp36m-linux_x86_64.whl size=376276 sha256=6ba898ccbbe7f0a67ab0e5505cb9766be22975ea2f9667b3ba42b10cccf8a585
  Stored in directory: /home/nbuser/.cache/pip/wheels/df/d0/79/37ea880586da03c620ca9ecd5

In [17]:
!pip install cvxopt

     |████████████████████████████████| 11.7MB 145kB/s eta 0:00:01    |█                               | 389kB 3.3MB/s eta 0:00:04     |████▌                           | 1.6MB 3.3MB/s eta 0:00:04      | 6.8MB 7.8MB/s eta 0:00:01     |████████████████████████████▎   | 10.3MB 7.8MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
!pip install pulp

     |████████████████████████████████| 40.6MB 44kB/s  eta 0:00:01    |█▎                              | 1.7MB 1.2MB/s eta 0:00:34     |████████                        | 10.2MB 3.2MB/s eta 0:00:10     |████████▎                       | 10.5MB 3.2MB/s eta 0:00:10     |████████████████▎               | 20.7MB 4.9MB/s eta 0:00:05     |██████████████████              | 22.9MB 4.9MB/s eta 0:00:04
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import cvxpy as cp

In [4]:
#First, need a function that picks elements from list L with probabilies probs
def pick_element_per_probs(L,probs):
    #make sure probs are normalized
    probs = [x / sum(probs) for x in probs]
    #pick a random number
    r = np.random.random()
    c = 0 #cumulative sum
    for i, prob in enumerate(probs):
        if r < prob + c:
            return L[i]
        c += prob

# Read in Data

### Option 1: Read in DR projections and DK salaries, and make sure names match

In [5]:
df = pd.read_csv('Data/DailyRotoPGAProjections_THE PLAYERS Championship.csv')
#del df['Salary']
df.head()

,Player,SlatePlayerID,Ownership,Sample,%:,Cut,Top 20,Top 5,Win,DK:,...,Finish Pts.1,Scoring Pts.1,Points.1,Value.1,Yahoo:,Salary.2,Finish Pts.2,Scoring Pts.2,Points.2,Value.2
0,Rory McIlroy,14421198,25.24,150,NaN,0.826750,0.553700,0.256750,0.081825,NaN,...,5.8478,79.20276,85.05,10.42,NaN,NaN,5.2590,74.72942,79.99,-3027.56
1,Jon Rahm,14421199,13.48,150,NaN,0.815275,0.523800,0.227475,0.065825,NaN,...,5.5163,76.45304,81.97,10.88,NaN,NaN,4.9214,72.07905,77.00,-2566.39
2,Justin Thomas,14421200,21.81,150,NaN,0.764025,0.430325,0.166250,0.045175,NaN,...,4.3448,70.71232,75.06,5.64,NaN,NaN,3.7515,66.14987,69.90,-2439.64
3,Hideki Matsuyama,14421211,19.96,150,NaN,0.740175,0.384625,0.135275,0.034450,NaN,...,3.8778,68.98552,72.86,12.82,NaN,NaN,3.2191,64.44148,67.66,-1116.98
4,Tommy Fleetwood,14421209,17.47,150,NaN,0.716900,0.346500,0.116250,0.028600,NaN,...,3.5399,66.57491,70.11,9.46,NaN,NaN,2.9198,62.05923,64.98,-1251.58


In [6]:
df.columns

Index(['Player', 'SlatePlayerID', 'Ownership', 'Sample', '%:', 'Cut', 'Top 20',
       'Top 5', 'Win', 'DK:', 'SD', 'Salary', 'Finish Pts', 'Scoring Pts',
       'Points', 'Value', 'FD:', 'SD.1', 'Salary.1', 'Finish Pts.1',
       'Scoring Pts.1', 'Points.1', 'Value.1', 'Yahoo:', 'Salary.2',
       'Finish Pts.2', 'Scoring Pts.2', 'Points.2', 'Value.2'],
      dtype='object')

In [7]:
df.SlatePlayerID = df.SlatePlayerID.astype('str')
df['Name + ID'] = df.Player + " (" + df.SlatePlayerID + ")"
df = df.sort_values(by='Salary',ascending=False)
df[['Player','Salary','Name + ID']].head(15)

,Player,Salary,Name + ID
0,Rory McIlroy,11700,Rory McIlroy (14421198)
1,Jon Rahm,11000,Jon Rahm (14421199)
2,Justin Thomas,10800,Justin Thomas (14421200)
29,Brooks Koepka,10200,Brooks Koepka (14421202)
21,Dustin Johnson,10000,Dustin Johnson (14421203)
10,Patrick Cantlay,9800,Patrick Cantlay (14421204)
9,Adam Scott,9600,Adam Scott (14421205)
6,Xander Schauffele,9400,Xander Schauffele (14421206)
5,Webb Simpson,9200,Webb Simpson (14421207)
8,Bryson DeChambeau,9100,Bryson DeChambeau (14421208)


In [8]:
#df[(df.Salary<6500)][['Player','Salary','Points']].sort_values(by='Points',ascending=False)

In [9]:
# from difflib import get_close_matches
# #df_DK = pd.read_csv('Data/DKSalaries Open 2019.csv')
# url = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=29&draftGroupId=29781'
# df_DK = pd.read_csv(url)
# df_DK.head()

In [10]:
# df['Name'] = ""
# for i in np.arange(len(df)):
#     curr_name = df['Player'].iloc[i]
#     try:
#         new_name = get_close_matches(curr_name,df_DK['Name'])[0]
#         if new_name != curr_name:
#             print('{} (from DR) changed to {} to match DK'.format(curr_name,new_name))
#     except:
#         new_name = 'No Name'
#         print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
#     df['Name'].iloc[i] = new_name
# #df2['Name'] = df2['Name'].apply(lambda x: get_close_matches(x,df2_sal['Name']))
# df.head(2)

In [11]:
# df = pd.merge(df,df_DK[['Name','Name + ID','Salary']], on='Name')
# df[['Player','Salary','Points','Ownership']].head()

In [12]:
#df.sort_values(by='Salary',ascending=False).head(10)

In [13]:
df.columns

Index(['Player', 'SlatePlayerID', 'Ownership', 'Sample', '%:', 'Cut', 'Top 20',
       'Top 5', 'Win', 'DK:', 'SD', 'Salary', 'Finish Pts', 'Scoring Pts',
       'Points', 'Value', 'FD:', 'SD.1', 'Salary.1', 'Finish Pts.1',
       'Scoring Pts.1', 'Points.1', 'Value.1', 'Yahoo:', 'Salary.2',
       'Finish Pts.2', 'Scoring Pts.2', 'Points.2', 'Value.2', 'Name + ID'],
      dtype='object')

# Optimize with Pulp

# Now Optimize in different ways

In [14]:
#OPTIONAL: Filter down data
df_all = df.copy()
#df[(df.Salary>=min_sal) & (df.Salary<= max_sal)][['Player','Salary']].sort_values(by='Salary')

In [15]:
df_all.head()

,Player,SlatePlayerID,Ownership,Sample,%:,Cut,Top 20,Top 5,Win,DK:,...,Scoring Pts.1,Points.1,Value.1,Yahoo:,Salary.2,Finish Pts.2,Scoring Pts.2,Points.2,Value.2,Name + ID
0,Rory McIlroy,14421198,25.24,150,NaN,0.826750,0.553700,0.256750,0.081825,NaN,...,79.20276,85.05,10.42,NaN,NaN,5.2590,74.72942,79.99,-3027.56,Rory McIlroy (14421198)
1,Jon Rahm,14421199,13.48,150,NaN,0.815275,0.523800,0.227475,0.065825,NaN,...,76.45304,81.97,10.88,NaN,NaN,4.9214,72.07905,77.00,-2566.39,Jon Rahm (14421199)
2,Justin Thomas,14421200,21.81,150,NaN,0.764025,0.430325,0.166250,0.045175,NaN,...,70.71232,75.06,5.64,NaN,NaN,3.7515,66.14987,69.90,-2439.64,Justin Thomas (14421200)
29,Brooks Koepka,14421202,13.93,150,NaN,0.594925,0.196650,0.050100,0.010075,NaN,...,55.62843,57.52,-7.24,NaN,NaN,1.3990,50.93915,52.34,-2056.39,Brooks Koepka (14421202)
21,Dustin Johnson,14421203,21.58,150,NaN,0.631825,0.239900,0.065050,0.013525,NaN,...,58.78089,61.08,-3.15,NaN,NaN,1.7469,54.07675,55.82,-1921.16,Dustin Johnson (14421203)


In [18]:
pen_vals = np.arange(0,1,.1)
for pen in pen_vals:
    df_now = df_all.copy()
    df_now = df_now[df_now.Salary<99999]
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.Points.values
    c = df_now.Cut.values
    T20 = df_now['Top 20'].values
    own = df_now.Ownership.values
    cap = 50000
    constraints = [cp.sum(x) == 6,
                 x*s <= cap]

    # Form objective.
    obj = cp.Maximize(x*(p-pen*own))

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Player'][x.value>.5].tolist()
    print('(pen = {:.1f}): '.format(pen),opt_team)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


SolverError: 

                    CVXPY needs additional software (a `mixed-integer solver`) to handle this model.
                    The web documentation
                        https://www.cvxpy.org/tutorial/advanced/index.html#mixed-integer-programs
                    reviews open-source and commercial options for mixed-integer solvers.

                    Quick fix: if you install the python package CVXOPT (pip install cvxopt),
                    then CVXPY can use the open-source mixed-integer solver `GLPK`.
                

In [23]:
caps = np.arange(110,35,-5)
for owncap in caps:
    df_now = df_all.copy()
    df_now = df_now[df_now.Salary<=12000]
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.Points.values
    c = df_now.Cut.values
    T20 = df_now['Top 20'].values
    own = df_now.Ownership.values
    cap = 50000
    constraints = [cp.sum(x) == 6,
                 x*s <= cap,
                  x*own <= owncap]

    # Form objective.
    obj = cp.Maximize(x*(p))

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Player'][x.value>.5].tolist()
    print('(own cap = {}%): '.format(owncap),opt_team)

(own cap = 110%):  ['Jon Rahm', 'Hideki Matsuyama', 'Collin Morikawa', 'Billy Horschel', 'Tyrrell Hatton', 'Abraham Ancer']
(own cap = 105%):  ['Jon Rahm', 'Hideki Matsuyama', 'Collin Morikawa', 'Billy Horschel', 'Tyrrell Hatton', 'Abraham Ancer']
(own cap = 100%):  ['Jon Rahm', 'Hideki Matsuyama', 'Collin Morikawa', 'Billy Horschel', 'Tyrrell Hatton', 'Abraham Ancer']
(own cap = 95%):  ['Jon Rahm', 'Hideki Matsuyama', 'Collin Morikawa', 'Tyrrell Hatton', 'Matthew Fitzpatrick', 'Abraham Ancer']
(own cap = 90%):  ['Jon Rahm', 'Hideki Matsuyama', 'Collin Morikawa', 'Billy Horschel', 'Matthew Fitzpatrick', 'Abraham Ancer']
(own cap = 85%):  ['Jon Rahm', 'Hideki Matsuyama', 'Collin Morikawa', 'Billy Horschel', 'Matthew Fitzpatrick', 'Abraham Ancer']
(own cap = 80%):  ['Jon Rahm', 'Hideki Matsuyama', 'Sungjae Im', 'Matthew Fitzpatrick', 'Abraham Ancer', 'Joel Dahmen']
(own cap = 75%):  ['Jon Rahm', 'Hideki Matsuyama', 'Sungjae Im', 'Billy Horschel', 'Abraham Ancer', 'Jim Furyk']
(own cap = 

In [27]:
#specify team settings

lock = ['Bryson DeChambeau','Viktor Hovland']
lock = ['Jon Rahm','Justin Thomas']
lock = []
lock_full_names = []
for i, p in enumerate(lock):
    lock_full_names.append(df[df.Player==p]['Name + ID'].iloc[0])
#print(lock_full_names)

to_remove = ['Charley Hoffman','Sam Ryder','Zach Johnson',
            'Jason Day','Sergio Garcia','Tiger Woods','Rafa Cabrera Bello',
            'Scott Piercy']
to_remove = ['Aaron Wise','Lucas Glover','Tom Lewis','Kevin Tway','Abraham Ancer','Jim Furyk',
            'Jason Day','Louis Oosthuizen','Billy Horschel','Paul Casey','Rickie Fowler']
max_sal = 9800
min_sal = 6000

numteams=8

In [28]:
#Now pick a penalty, and find 1st, 2nd, 3rd, ... opt teams
#df = df_all.copy()
df_now = df_all.copy()

sal_used = df_now[df_now.Player.isin(lock)].Salary.sum()
sal_left = 50000-sal_used
players_left = 6-len(lock)

#Filter down remaining players by sal
df_now = df_now[(df_now.Salary>=min_sal) &
               (df_now.Salary<=max_sal)]
df_now = df_now[~df_now.Player.isin(lock)]
df_now = df_now[~df_now.Player.isin(to_remove)]

penalty = 0.4
opt_teams = []
for i in np.arange(numteams): #number of teams
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.Points.values
    c = df_now.Cut.values
    T20 = df_now['Top 20'].values
    own = df_now.Ownership.values
    cap = sal_left
    constraints = [cp.sum(x) == players_left,
                 x*s <= cap]

    # Form objective.
    obj = cp.Maximize(x*(p-penalty*own))

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Player'][x.value>.5].tolist()
    print('Team {} (obj = {:.2f}): '.format(i+1,prob.value),opt_team)
    opt_teams.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Player.isin(opt_team)]


Team 1 (obj = 373.48):  ['Xander Schauffele', 'Hideki Matsuyama', 'Sungjae Im', 'Patrick Reed', 'Matthew Fitzpatrick', 'Ryan Moore']
Team 2 (obj = 359.32):  ['Webb Simpson', 'Bryson DeChambeau', 'Tommy Fleetwood', 'Henrik Stenson', 'Scottie Scheffler', 'Brandt Snedeker']
Team 3 (obj = 346.81):  ['Patrick Cantlay', 'Adam Scott', 'Gary Woodland', 'Viktor Hovland', 'Tyrrell Hatton', 'Chez Reavie']
Team 4 (obj = 333.80):  ['Collin Morikawa', 'Byeong Hun An', 'Matt Kuchar', 'Shane Lowry', 'Adam Hadwin', 'Vaughn Taylor']
Team 5 (obj = 328.42):  ['Justin Rose', 'Marc Leishman', 'Brendon Todd', 'Corey Conners', 'Rory Sabbatini', 'Nick Taylor']
Team 6 (obj = 321.97):  ['Harris English', 'Kevin Na', 'Harold Varner III', 'Tom Hoge', 'Brian Harman', 'Bud Cauley']
Team 7 (obj = 316.62):  ['Sergio Garcia', 'Kevin Kisner', 'Erik Van Rooyen', 'Daniel Berger', 'Carlos Ortiz', 'Christiaan Bezuidenhout']
Team 8 (obj = 312.90):  ['Tony Finau', 'Rafa Cabrera Bello', 'Russell Knox', 'Joel Dahmen', 'J.T. Pos

In [29]:
#Find opt teams with max own cap

df=df_all.copy()

df_now = df.copy()
df_now = df_now[(df_now.Salary>=min_sal) & (df_now.Salary<=max_sal)]

sal_used = df_now[df_now.Player.isin(lock)].Salary.sum()
own_used = df_now[df_now.Player.isin(lock)].Ownership.sum()
#print(own_used)
sal_left = 50000-sal_used
players_left = 6-len(lock)
df_now = df_now[~df_now.Player.isin(lock)]
df_now = df_now[~df_now.Player.isin(to_remove)]


own_cap = 68 - own_used
#print(own_cap)
opt_teams_cap = []
for i in np.arange(numteams): #number of teams
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.Points.values
    c = df_now.Cut.values
    T20 = df_now['Top 20'].values
    own = df_now.Ownership.values
    cap = sal_left
    constraints = [cp.sum(x) == players_left,
                 x*s <= cap,
                  x*own <= own_cap]

    # Form objective.
    obj = cp.Maximize(x*(p))

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Player'][x.value>.5].tolist()
    print('Team {} (obj = {:.2f}): '.format(i+1,prob.value),opt_team)
    opt_teams_cap.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Player.isin(opt_team)]

Team 1 (obj = 399.28):  ['Tommy Fleetwood', 'Hideki Matsuyama', 'Sungjae Im', 'Patrick Reed', 'Matthew Fitzpatrick', 'Ryan Moore']
Team 2 (obj = 384.01):  ['Xander Schauffele', 'Webb Simpson', 'Bryson DeChambeau', 'Henrik Stenson', 'Scottie Scheffler', 'Vaughn Taylor']
Team 3 (obj = 372.90):  ['Patrick Cantlay', 'Adam Scott', 'Collin Morikawa', 'Viktor Hovland', 'Brandt Snedeker', 'Chez Reavie']
Team 4 (obj = 362.30):  ['Justin Rose', 'Gary Woodland', 'Byeong Hun An', 'Matt Kuchar', 'Shane Lowry', 'Tyrrell Hatton']
Team 5 (obj = 346.92):  ['Tony Finau', 'Sergio Garcia', 'Marc Leishman', 'Daniel Berger', 'Adam Hadwin', 'Joel Dahmen']
Team 6 (obj = 330.83):  ['Kevin Kisner', 'Ian Poulter', 'Harris English', 'Corey Conners', 'Rory Sabbatini', 'Nick Taylor']
Team 7 (obj = 325.37):  ['Brendon Todd', 'Kevin Na', 'Harold Varner III', 'Tom Hoge', 'Brian Harman', 'Bud Cauley']
Team 8 (obj = 318.94):  ['Erik Van Rooyen', 'Christiaan Bezuidenhout', 'Jason Kokrak', 'Max Homa', 'Ryan Palmer', 'J.T.

In [30]:
#Who's in one opt but not the other
for num_teams in np.arange(1,6):
    print('In top {} teams:'.format(num_teams))
    pen_not_cap = set(np.hstack(opt_teams[:num_teams])) - set(np.hstack(opt_teams_cap[:num_teams]))
    cap_not_pen = set(np.hstack(opt_teams_cap[:num_teams])) - set(np.hstack(opt_teams[:num_teams]))
    print('\tIn pen, not cap',pen_not_cap)
    print('\tIn cap, not pen:',cap_not_pen)

In top 1 teams:
	In pen, not cap {'Xander Schauffele'}
	In cap, not pen: {'Tommy Fleetwood'}
In top 2 teams:
	In pen, not cap {'Brandt Snedeker'}
	In cap, not pen: {'Vaughn Taylor'}
In top 3 teams:
	In pen, not cap {'Tyrrell Hatton', 'Gary Woodland'}
	In cap, not pen: {'Collin Morikawa', 'Vaughn Taylor'}
In top 4 teams:
	In pen, not cap {'Adam Hadwin'}
	In cap, not pen: {'Justin Rose'}
In top 5 teams:
	In pen, not cap {'Nick Taylor', 'Corey Conners', 'Rory Sabbatini', 'Brendon Todd'}
	In cap, not pen: {'Joel Dahmen', 'Tony Finau', 'Sergio Garcia', 'Daniel Berger'}


In [59]:
#pick the num of teams to use

## Generate lineups!

In [31]:
### Assign exposures for top j teams
num_teams_deep = 3
own_gap = 5
exposures = list(np.arange(50,50-own_gap*num_teams_deep,-own_gap))
if len(exposures)<len(opt_teams_cap):
    exposures = exposures + [0]*(len(opt_teams)-len(exposures))
exposures

[50, 45, 40, 0, 0, 0, 0, 0]

In [32]:
#Create df to hold player pool
df_teams = df[['Name + ID','Player','Salary','Ownership']]
df_teams.sort_values(by='Salary',ascending=False,inplace=True)
df_teams['Exposure'] = 0
df_teams['Actual Exposure'] = 0
df_teams.head()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

,Name + ID,Player,Salary,Ownership,Exposure,Actual Exposure
0,Rory McIlroy (14421198),Rory McIlroy,11700,25.24,0,0
1,Jon Rahm (14421199),Jon Rahm,11000,13.48,0,0
2,Justin Thomas (14421200),Justin Thomas,10800,21.81,0,0
29,Brooks Koepka (14421202),Brooks Koepka,10200,13.93,0,0
21,Dustin Johnson (14421203),Dustin Johnson,10000,21.58,0,0


In [33]:
#Add in exposures for players in player pool
for i, team in enumerate(opt_teams):
    df_teams.Exposure = np.where(df_teams.Player.isin(team),exposures[i],df_teams.Exposure)
    
#OPTIONAL: Add or adjust players
#more_team = {'Francesco Molinari':20,'Henrik Stenson':20,'Lucas Glover':20}
#more_team = {'Sungjae Im':35}
# more_team = {}
# #more_team['Sungjae Im']=100
# more_players = {}
# for p in more_players:
#     more_team[p] = 40
#     _

#Add teams from other method
num_teams = len([e for e in exposures if e>0])
pen_not_cap = set(np.hstack(opt_teams[:num_teams])) - set(np.hstack(opt_teams_cap[:num_teams]))
cap_not_pen = set(np.hstack(opt_teams_cap[:num_teams])) - set(np.hstack(opt_teams[:num_teams]))
more_players={p:40 for p in pen_not_cap}
more_players = {}
more_team=['Kevin Na','Adam Hadwin','Sebastian Munoz']
#more_players['Cameron Champ'] = 39
#more_players['Henrik Stenson']
for p in more_team:
    more_players[p]=30
for p in more_players.keys():
    df_teams.Exposure = np.where(df_teams.Player==p,more_players[p],df_teams.Exposure)
    
df_teams = df_teams[df_teams.Exposure > 0].reset_index(drop=True)
df_teams

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Name + ID,Player,Salary,Ownership,Exposure,Actual Exposure
0,Patrick Cantlay (14421204),Patrick Cantlay,9800,19.53,40,0
1,Adam Scott (14421205),Adam Scott,9600,18.84,40,0
2,Xander Schauffele (14421206),Xander Schauffele,9400,13.93,50,0
3,Webb Simpson (14421207),Webb Simpson,9200,15.19,45,0
4,Bryson DeChambeau (14421208),Bryson DeChambeau,9100,21.61,45,0
5,Tommy Fleetwood (14421209),Tommy Fleetwood,9000,17.47,45,0
6,Hideki Matsuyama (14421211),Hideki Matsuyama,8800,19.96,50,0
7,Sungjae Im (14421213),Sungjae Im,8600,11.88,50,0
8,Patrick Reed (14421214),Patrick Reed,8500,11.19,50,0
9,Gary Woodland (14421216),Gary Woodland,8300,14.62,40,0


In [34]:
sal_used
players_left
min_sal, max_sal

(6000, 9800)

In [41]:
#Now start churning
#Now set parameters for number of teams and salary limits
num_teams = 45
min_sal = 49300
max_sal = 49900

#initialize df_lineups, which will be written to csv
df_lineups = pd.DataFrame(columns=['G1','G2','G3','G4','G5','G6'])
df_lineups

#Reset df_teams
df_teams['Actual Exposure'] = 0

m = 0 #Completed teams so far
while (m < num_teams):
    #pick 6 players per probabilties
    curr_team = []
    df_now = df_teams.copy()
    players = df_now['Name + ID'].tolist()
    probs = df_now['Exposure'].tolist()
    for i in range(players_left):
        p = pick_element_per_probs(players,probs)
        curr_team.append(p)
        #prit('picked player {}'.format(p))
        df_now = df_now[df_now['Name + ID']!=p]
        players = df_now['Name + ID'].tolist()
        probs = df_now['Exposure'].tolist()
        
    #Now curr_team is list of 6 names.  Check if salary matches
    df_now = df_teams[df_teams['Name + ID'].isin(curr_team)]
    total_own = df_now.Ownership.sum()
    #total_own = 0 #use for small field where ownership is bonkers
    total_sal = df_now.Salary.sum() + sal_used
    ##print(total_sal)
    if (total_sal < min_sal or total_sal > max_sal or total_own > 75):
        #print('Team eliminated {}, total_sal = {}, total_own = {}'.format('team',total_sal,total_own))
        continue
        
    #Put team in order of decreasing salary
    df_now.sort_values(by='Salary',ascending=False,inplace=True)
    curr_team = lock_full_names + df_now['Name + ID'].tolist()
    #print('New Team',curr_team)
    
    #Check if team has shown up before in df_lineups
    df3 = (df_lineups == np.array(curr_team))
    old_team = df3.transpose().all().any() #True if team is already in df_lineups
    if old_team:
        #print('Team already exists')
        continue
        
    #OK, so now we have a new team!  Add it to df_lineups
    df_lineups.loc[m] = curr_team
    #Also update 'Actual Exposure' in df_teams
    new_players = np.where(df_teams['Name + ID'].isin(curr_team),1,0)
    df_teams['Actual Exposure'] = df_teams['Actual Exposure'] + new_players
    
    #Finally increment m
    m += 1
    if m%10==0:
        print('m = {}'.format(m))

    #print('Team {}, sal = {}'.format(m,total_sal))
    
df_teams[['Name + ID','Actual Exposure','Exposure','Salary']]

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


m = 10
m = 20
m = 30
m = 40


,Name + ID,Actual Exposure,Exposure,Salary
0,Patrick Cantlay (14421204),19,40,9800
1,Adam Scott (14421205),13,40,9600
2,Xander Schauffele (14421206),20,50,9400
3,Webb Simpson (14421207),19,45,9200
4,Bryson DeChambeau (14421208),11,45,9100
5,Tommy Fleetwood (14421209),13,45,9000
6,Hideki Matsuyama (14421211),14,50,8800
7,Sungjae Im (14421213),14,50,8600
8,Patrick Reed (14421214),7,50,8500
9,Gary Woodland (14421216),15,40,8300


In [42]:
for c in df_lineups.columns:
    df_lineups.rename(columns={c:'G'},inplace=True)

In [43]:
#If you're good with actual exposure, then write it!
df_lineups.to_csv('Lineups/DK Entires.csv',index=False)

In [68]:
df_lineups.head()

,G,G,G,G,G,G
0,Rory McIlroy (14421198),Bryson DeChambeau (14421208),Collin Morikawa (14421222),Scottie Scheffler (14421231),Jim Furyk (14421270),Joel Dahmen (14421285)
1,Rory McIlroy (14421198),Sungjae Im (14421213),Patrick Reed (14421214),Tyrrell Hatton (14421232),Cameron Champ (14421264),Brian Harman (14421299)
2,Rory McIlroy (14421198),Tommy Fleetwood (14421209),Patrick Reed (14421214),Tyrrell Hatton (14421232),Cameron Champ (14421264),Bud Cauley (14421314)
3,Rory McIlroy (14421198),Sungjae Im (14421213),Collin Morikawa (14421222),Scottie Scheffler (14421231),Billy Horschel (14421230),Jim Furyk (14421270)
4,Rory McIlroy (14421198),Bryson DeChambeau (14421208),Tommy Fleetwood (14421209),Cameron Champ (14421264),Corey Conners (14421273),Bud Cauley (14421314)


# FD

In [69]:
df2 = pd.read_csv('Data/DailyRotoPGAProjections_Shriners Hospital for Children Open FD.csv')
#del df['Salary']
df2.head()

,Player,SlatePlayerID,Ownership,Sample,%:,Cut,Top 20,Top 5,Win,DK:,...,Scoring Pts,Points,Value,FD:,SD.1,Salary.1,Finish Pts.1,Scoring Pts.1,Points.1,Value.1
0,Brooks Koepka,38974-78367,16.54,150,NaN,0.810825,0.511350,0.214650,0.063875,NaN,...,83.711385,91.11,11.44,NaN,37.787783,12200.0,5.305312,88.744152,94.05,12.77
1,Patrick Cantlay,38974-79435,22.64,150,NaN,0.823925,0.522425,0.213650,0.060775,NaN,...,83.614958,91.05,12.53,NaN,36.715761,11800.0,5.375305,89.097241,94.47,14.19
2,Adam Scott,38974-78780,18.98,150,NaN,0.797225,0.471825,0.185025,0.050925,NaN,...,81.334255,88.07,11.53,NaN,37.297350,11400.0,4.880586,86.375940,91.26,13.03
3,Webb Simpson,38974-78804,13.54,150,NaN,0.786125,0.445225,0.165175,0.043200,NaN,...,79.074530,85.08,9.68,NaN,37.045458,11300.0,4.364660,83.958062,88.32,11.22
4,Hideki Matsuyama,38974-78508,17.87,150,NaN,0.765225,0.421750,0.153825,0.040150,NaN,...,78.440117,84.09,9.54,NaN,37.204530,11100.0,4.101694,83.068275,87.17,10.97


In [70]:
df2.columns

Index(['Player', 'SlatePlayerID', 'Ownership', 'Sample', '%:', 'Cut', 'Top 20',
       'Top 5', 'Win', 'DK:', 'SD', 'Salary', 'Finish Pts', 'Scoring Pts',
       'Points', 'Value', 'FD:', 'SD.1', 'Salary.1', 'Finish Pts.1',
       'Scoring Pts.1', 'Points.1', 'Value.1'],
      dtype='object')

In [71]:
#DF salary is in Salary.1 column
del df2['Salary']
df2.rename(columns={'Salary.1':'Salary'},inplace=True)
df2.sort_values(by='Salary',ascending=False).head()

,Player,SlatePlayerID,Ownership,Sample,%:,Cut,Top 20,Top 5,Win,DK:,...,Scoring Pts,Points,Value,FD:,SD.1,Salary,Finish Pts.1,Scoring Pts.1,Points.1,Value.1
0,Brooks Koepka,38974-78367,16.54,150,NaN,0.810825,0.511350,0.214650,0.063875,NaN,...,83.711385,91.11,11.44,NaN,37.787783,12200.0,5.305312,88.744152,94.05,12.77
1,Patrick Cantlay,38974-79435,22.64,150,NaN,0.823925,0.522425,0.213650,0.060775,NaN,...,83.614958,91.05,12.53,NaN,36.715761,11800.0,5.375305,89.097241,94.47,14.19
8,Bryson DeChambeau,38974-78005,12.43,150,NaN,0.722925,0.337950,0.108250,0.024250,NaN,...,74.359083,79.14,4.61,NaN,37.008679,11600.0,3.447313,78.506735,81.95,5.85
2,Adam Scott,38974-78780,18.98,150,NaN,0.797225,0.471825,0.185025,0.050925,NaN,...,81.334255,88.07,11.53,NaN,37.297350,11400.0,4.880586,86.375940,91.26,13.03
3,Webb Simpson,38974-78804,13.54,150,NaN,0.786125,0.445225,0.165175,0.043200,NaN,...,79.074530,85.08,9.68,NaN,37.045458,11300.0,4.364660,83.958062,88.32,11.22


In [72]:
#Add field with the player:SlatePlayerID that will go into lineup csvs
df2['Name + ID'] =  df2.SlatePlayerID  + ":" + df2.Player

In [73]:
#specify team settings

lock = ['Bryson DeChambeau','Viktor Hovland']
lock = ['Marc Leishman','Bubba Watson']
lock = ['Patrick Cantlay','Dustin Johnson']
#lock = []

to_remove = ['Charley Hoffman','Sam Ryder','Zach Johnson',
            'Jason Day','Sergio Garcia','Tiger Woods','Rafa Cabrera Bello',
            'Scott Piercy']
to_remove = ['Kevin Chappell','Matt Jones']

max_sal = 910200

In [74]:
#Now pick a penalty, and find 1st, 2nd, 3rd, ... opt teams
#df = df_all.copy()
df_now = df2.copy()

lock_full_names = []
for i, p in enumerate(lock):
    lock_full_names.append(df_now[df_now.Player==p]['Name + ID'].iloc[0])
sal_used = df_now[df_now.Player.isin(lock)].Salary.sum()
sal_left = 60000-sal_used
players_left = 6-len(lock)
print(lock_full_names)

#Filter down remaining players by sal
df_now = df_now[(df_now.Salary<=max_sal)]
df_now = df_now[~df_now.Player.isin(lock)]
df_now = df_now[~df_now.Player.isin(to_remove)]

penalty = 0.3
opt_teams = []
for i in np.arange(5): #number of teams
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.Points.values
    c = df_now.Cut.values
    T20 = df_now['Top 20'].values
    own = df_now.Ownership.values
    cap = sal_left
    constraints = [cp.sum(x) == players_left,
                 x*s <= cap]

    # Form objective.
    obj = cp.Maximize(x*(p-penalty*own))

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Player'][x.value>.5].tolist()
    print('Team {} (obj = {:.2f}): '.format(i+1,prob.value),opt_team)
    opt_teams.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Player.isin(opt_team)]


IndexError: single positional indexer is out-of-bounds

In [ ]:
#Find opt teams with max own cap

df_now = df2.copy()

lock_full_names = []
for i, p in enumerate(lock):
    lock_full_names.append(df_now[df_now.Player==p]['SlatePlayerID'].iloc[0]+":"+p)
sal_used = df_now[df_now.Player.isin(lock)].Salary.sum()
own_used = df_now[df_now.Player.isin(lock)].Ownership.sum()
sal_left = 60000-sal_used
players_left = 6-len(lock)
print(lock_full_names)

#Filter down remaining players by sal
df_now = df_now[(df_now.Salary<=max_sal)]
df_now = df_now[~df_now.Player.isin(lock)]
df_now = df_now[~df_now.Player.isin(to_remove)]


own_cap = 100 - own_used
#print(own_cap)
opt_teams_cap = []
for i in np.arange(11): #number of teams
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    p = df_now.Points.values
    c = df_now.Cut.values
    T20 = df_now['Top 20'].values
    own = df_now.Ownership.values
    cap = sal_left
    constraints = [cp.sum(x) == players_left,
                 x*s <= cap,
                  x*own <= own_cap]

    # Form objective.
    obj = cp.Maximize(x*(p))

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()  # Returns the optimal value
    opt_team = df_now['Player'][x.value>.5].tolist()
    print('Team {} (obj = {:.2f}): '.format(i+1,prob.value),opt_team)
    opt_teams_cap.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Player.isin(opt_team)]

In [ ]:
### Assign exposures for top j teams
exposures = [50,45]
if len(exposures)<len(opt_teams):
    exposures = exposures + [0]*(len(opt_teams)-len(exposures))
exposures

In [ ]:
#Create df to hold player pool
df_teams = df2[['Name + ID','Player','Salary','Ownership']]
df_teams.sort_values(by='Salary',ascending=False,inplace=True)
df_teams['Exposure'] = 0
df_teams['Actual Exposure'] = 0
df_teams.head()

In [ ]:
#Add in exposures for players in player pool
for i, team in enumerate(opt_teams):
    df_teams.Exposure = np.where(df_teams.Player.isin(team),exposures[i],df_teams.Exposure)
    
#OPTIONAL: Add or adjust players
#more_team = {'Francesco Molinari':20,'Henrik Stenson':20,'Lucas Glover':20}
more_team = {'Xinjun Zhang':35}
#more_team = {}
for p in more_team.keys():
    df_teams.Exposure = np.where(df_teams.Player==p,more_team[p],df_teams.Exposure)
    
df_teams = df_teams[df_teams.Exposure > 0].reset_index(drop=True)
df_teams

In [ ]:
#initialize df_lineups, which will be written to csv
df_lineups = pd.DataFrame(columns=['G1','G2','G3','G4','G5','G6'])
df_lineups

In [ ]:
#Now start churning
#Now set parameters for number of teams and salary limits
num_teams = 32
min_sal = 58500
max_sal = 59800

#Reset df_teams
df_teams['Actual Exposure'] = 0

m = 0 #Completed teams so far
while (m < num_teams):
    #pick 6 players per probabilties
    curr_team = []
    df_now = df_teams.copy()
    players = df_now['Name + ID'].tolist()
    probs = df_now['Exposure'].tolist()
    for i in range(players_left):
        p = pick_element_per_probs(players,probs)
        curr_team.append(p)
        #prit('picked player {}'.format(p))
        df_now = df_now[df_now['Name + ID']!=p]
        players = df_now['Name + ID'].tolist()
        probs = df_now['Exposure'].tolist()
        
    #Now curr_team is list of 6 names.  Check if salary matches
    df_now = df_teams[df_teams['Name + ID'].isin(curr_team)]
    total_own = df_now.Ownership.sum()
    total_sal = df_now.Salary.sum() + sal_used
    ##print(total_sal)
    if (total_sal < min_sal or total_sal > max_sal or total_own > 85):
        continue
        
    #Put team in order of decreasing salary
    df_now.sort_values(by='Salary',ascending=False,inplace=True)
    curr_team = lock_full_names + df_now['Name + ID'].tolist()
    
    #Check if team has shown up before in df_lineups
    df3 = (df_lineups == np.array(curr_team))
    old_team = df3.transpose().all().any() #True if team is already in df_lineups
    if old_team:
        continue
        
    #OK, so now we have a new team!  Add it to df_lineups
    df_lineups.loc[m] = curr_team
    #Also update 'Actual Exposure' in df_teams
    new_players = np.where(df_teams['Name + ID'].isin(curr_team),1,0)
    df_teams['Actual Exposure'] = df_teams['Actual Exposure'] + new_players
    
    #Finally increment m
    m += 1
    #print('m = {}'.format(m))
    #print('Team {}, sal = {}'.format(m,total_sal))
    
df_teams[['Player','Name + ID','Actual Exposure']]

In [ ]:
#If you're good with actual exposure, then write it!
df_lineups.to_csv('Lineups/FD Entires.csv')

In [ ]:
df_lineups